In [ ]:
from tabular_anonymizer import DataFrameAnonymizer
import pandas as pd

In [ ]:
file1 = "adult.csv"
df = pd.read_csv(file1, sep=",", index_col=0)
df.reset_index()
df.index = range(len(df))
df.head()

In [ ]:
# Drop some columns and rows
# Keep only 1000 rows
df = df.loc[0:999,]
print(df.shape)
# Drop some columns
df = df.drop(columns = ["fnlwgt", "workclass", "education-num", "occupation", "race", "native-country"])
df.head()

In [ ]:
# Setup Sensitive attributes you dont want to alter
sensitive_columns = ['label']
# Avg columns
avg_columns = ['capital-gain', 'capital-loss']

# Run tabular_anonymizer
# Set k
k = 5

# List of quasi-identifiers for checking k-anonymity
quasi_identifiers = ['age',	'education', 'marital-status', 'relationship',
	'sex', 'capital-gain', 'capital-loss', 'hours-per-week']

print("Run tabular_anonymizer. Sensitive columns: ", sensitive_columns, ", k=", k)
p = DataFrameAnonymizer(df, sensitive_columns, avg_columns=avg_columns, format_to_str=True)

# New anonymized dataframe is formed
df_anonymized = p.anonymize_k_anonymity(k=k)
df_anonymized.head()

In [ ]:
def is_k_anonymous(df, k, quasi_identifiers, count_column debug=False):
    for index, row in df.iterrows():
        # If row count < k, search for other identical rows with different sensitive attribute
        if row[count_column] < k:
            query = ' & '.join([f'`{col}` == {row[col]}' for col in quasi_identifiers])
            if debug:
                print(query)
            rows = df.query(query)
            if debug:
                print(rows)
            if sum(rows[count_column]) < k:
                return False
    return True

# Miksi query ei toimi?


In [ ]:
columns_to_check = ['sex']

is_k_anonymous(df_anonymized, 5, columns_to_check, 'label_count')

Cells below are for testing only (i.e. trash)

In [ ]:
#df_anonymized['sex']
df_anonymized.loc[0, 'sex'] == ['Male']

In [ ]:
df_anonymized.query("`sex` == ['Male']")

In [ ]:
# Only numeric columns
num_columns = ['capital-gain', 'capital-loss']

is_k_anonymous(df_anonymized, 5, num_columns, 'label_count')

In [ ]:
# Original
#def isKAnonymized(df, k):
#    for index, row in df.iterrows():
#        query = ' & '.join([f'{col} == "{row[col]}"' for col in df.columns])
#        rows = df.query(query)
#        if rows.shape[0] < k:
#            return False
#    return True